<a href="https://colab.research.google.com/github/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_analise_mercado_audi_vs_porsche/projeto_audi_porsche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importando todas bibliotecas necessárias

import requests
import zipfile
import io
import os
import glob
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [3]:
# Carregando arquivo direto do site

# URL do arquivo .zip
url = 'https://dados.transportes.gov.br/dataset/12686da0-3d71-4499-b432-d270f785c907/resource/ee0d392c-c467-4002-884d-e89bd1f989f1/download/i_frota_por_uf_municipio_marca_e_modelo_ano_abril_2025.zip'
# Faz o download do arquivo com barra de progresso
print("Iniciando download do arquivo...")
response = requests.get(url, stream=True)  # stream=True para download em partes

# Verifica se o download foi bem-sucedido
if response.status_code == 200:
    # Obtém o tamanho total do arquivo (em bytes)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte

    # Cria um buffer para armazenar o conteúdo
    file_buffer = io.BytesIO()

    # Barra de progresso para o download
    with tqdm(total=total_size, unit='iB', unit_scale=True, desc="Download") as pbar:
        for data in response.iter_content(block_size):
            file_buffer.write(data)
            pbar.update(len(data))  # Atualiza a barra de progresso

    print("\nDownload concluído com sucesso!")

    # Prepara para extrair o arquivo
    file_buffer.seek(0)  # Volta ao início do buffer

    # Extrai o conteúdo do arquivo .zip com barra de progresso
    print("\nExtraindo arquivos...")
    with zipfile.ZipFile(file_buffer) as zip_file:
        # Obtém lista de arquivos para extração
        file_list = zip_file.infolist()

        # Barra de progresso para extração
        for file in tqdm(file_list, desc="Extraindo"):
            zip_file.extract(file, 'dados_extraidos')

    print("Extração concluída com sucesso!")


Iniciando download do arquivo...


Download: 100%|██████████| 128M/128M [02:50<00:00, 754kiB/s]



Download concluído com sucesso!

Extraindo arquivos...


Extraindo: 100%|██████████| 1/1 [00:11<00:00, 11.08s/it]

Extração concluída com sucesso!


In [4]:
# Busca o arquivo na pasta
arquivos = glob.glob(os.path.join('dados_extraidos', '*'))
if arquivos:
    caminho_original = arquivos[0]  # Pega o primeiro
    novo_nome = os.path.join('dados_extraidos', 'base_de_dados.csv') # renomeia e transforma em .csv

    os.rename(caminho_original, novo_nome)
    print(f"Arquivo renomeado para: {novo_nome}")
else:
    print("Nenhum arquivo TXT encontrado na pasta.")

Arquivo renomeado para: dados_extraidos/base_de_dados.csv


In [5]:
# Analisando o arquivo

# Caminho do arquivo
caminho_arquivo = os.path.join('dados_extraidos', 'base_de_dados.csv')

# Verifica se o arquivo existe
if os.path.exists(caminho_arquivo):
    print("Carregando o arquivo...")

    try:
        # Configuração do pandas para mostrar todas as colunas no info()
        pd.set_option('display.max_columns', None)

        # Carrega o arquivo em chunks (pedaços) para arquivos grandes
        chunks = pd.read_csv(caminho_arquivo,
                           encoding='utf-8',
                           delimiter=';',
                           chunksize=10000,
                           low_memory=False)

        # Concatena todos os chunks em um único DataFrame
        df = pd.concat(chunks)

        # Mostra apenas as informações do DataFrame
        print("\nInformações completas do DataFrame:")
        df.info(verbose=True, memory_usage='deep', show_counts=True)

    except Exception as e:
        print(f"\nErro ao ler o arquivo: {e}")
else:
    print("Arquivo não encontrado. Verifique o caminho.")

Carregando o arquivo...

Informações completas do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21446674 entries, 0 to 21446673
Data columns (total 5 columns):
 #   Column                      Non-Null Count     Dtype  
---  ------                      --------------     -----  
 0   UF                          21446674 non-null  object 
 1   Município                   21446674 non-null  object 
 2   Marca Modelo                21446674 non-null  object 
 3   Ano Fabricação Veículo CRV  21446674 non-null  object 
 4   Qtd. Veículos               21446674 non-null  float64
dtypes: float64(1), object(4)
memory usage: 5.6 GB


In [6]:
# Verificando as primeiras linhas
df.head()

,UF,Município,Marca Modelo,Ano Fabricação Veículo CRV,Qtd. Veículos
0,ACRE,ACRELANDIA,AGRALE/13000,2009,1.0
1,ACRE,ACRELANDIA,AGRALE/1800,1989,1.0
2,ACRE,ACRELANDIA,AGRALE/1800,1990,1.0
3,ACRE,ACRELANDIA,AGRALE/1800D RD,1989,1.0
4,ACRE,ACRELANDIA,AGRALE/1800D RD,1990,2.0


In [7]:
# Verificando as ultimas linhas
df.tail()

,UF,Município,Marca Modelo,Ano Fabricação Veículo CRV,Qtd. Veículos
21446669,TOCANTINS,XAMBIOA,YAMAHA/YS150 FAZER SED,2014,1.0
21446670,TOCANTINS,XAMBIOA,YAMAHA/YS150 FAZER SED,2015,1.0
21446671,TOCANTINS,XAMBIOA,YAMAHA/YS150 FAZER SED,2018,1.0
21446672,TOCANTINS,XAMBIOA,YAMAHA/YS150 FAZER SED,2019,1.0
21446673,TOCANTINS,XAMBIOA,YAMAHA/YS150 FAZER SED,2020,1.0


In [8]:
df.describe()

,Qtd. Veículos
count,2.144667e+07
mean,5.983354e+00
std,6.653207e+01
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,3.000000e+00
max,7.911400e+04


In [9]:
df.describe(include='object')

,UF,Município,Marca Modelo,Ano Fabricação Veículo CRV
count,21446674,21446674,21446674,21446674
unique,30,5291,37784,129
top,SAO PAULO,SAO PAULO,VW/KOMBI,2013
freq,4308689,78181,96223,891865


In [11]:
# Confirmando que não há dados nulos

nulos = df.isnull().sum()
print(nulos)

UF                            0
Município                     0
Marca Modelo                  0
Ano Fabricação Veículo CRV    0
Qtd. Veículos                 0
dtype: int64


In [21]:
# Criando novo dataframe somente com veículo da marca AUDI fabricados em 2024


df_audi_2024 = df[df['Marca Modelo'].str.contains('AUDI', case=False, na=False) ]

df_audi_2024 = df_audi_2024[df_audi_2024['Ano Fabricação Veículo CRV'].str.contains('2024')]

df_audi_2024.head()


,UF,Município,Marca Modelo,Ano Fabricação Veículo CRV,Qtd. Veículos
105726,ALAGOAS,CORURIPE,I/AUDI Q5 SB P-HIBRIDO,2024,1.0
146267,ALAGOAS,MACEIO,AUDI/Q3 2.0 TFSI,2024,1.0
146270,ALAGOAS,MACEIO,AUDI/Q3 SPB 2.0 TFSI,2024,6.0
154366,ALAGOAS,MACEIO,I/AUDI A3 SD HIBRIDO,2024,3.0
154367,ALAGOAS,MACEIO,I/AUDI A3 SEDAN 2.0,2024,2.0


In [24]:
df_audi_2024.describe(include=object)

,UF,Município,Marca Modelo,Ano Fabricação Veículo CRV
count,1418,1418,1418,1418
unique,22,448,38,1
top,SAO PAULO,BRASILIA,I/AUDI Q5 2.0 TFSI,2024
freq,503,27,152,1418


In [22]:
# Criando novo dataframe somente com veículo da marca PORSCHE fabricados em 2024


df_porsche_2024 = df[df['Marca Modelo'].str.contains('PORSCHE', case=False, na=False) ]

df_porsche_2024 = df_porsche_2024[df_porsche_2024['Ano Fabricação Veículo CRV'].str.contains('2024')]

df_porsche_2024.head()


,UF,Município,Marca Modelo,Ano Fabricação Veículo CRV,Qtd. Veículos
157726,ALAGOAS,MACEIO,I/PORSCHE 911 CARRERA,2024,1.0
157739,ALAGOAS,MACEIO,I/PORSCHE BOXSTER,2024,1.0
157746,ALAGOAS,MACEIO,I/PORSCHE CAYENNE CP PHE,2024,3.0
157751,ALAGOAS,MACEIO,I/PORSCHE CAYENNE PHEV,2024,1.0
157757,ALAGOAS,MACEIO,I/PORSCHE CAYENNE SCPHEV,2024,2.0


In [25]:
df_porsche_2024.describe(include=object)

,UF,Município,Marca Modelo,Ano Fabricação Veículo CRV
count,2072,2072,2072,2072
unique,24,543,48,1
top,SAO PAULO,SAO PAULO,I/PORSCHE MACAN,2024
freq,775,43,182,2072


In [26]:
# Criar arquivos .csv

df_audi_2024.to_csv('audi_2024.csv', index=False)
df_porsche_2024.to_csv('porsche_2024.csv', index=False)